## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
device

device(type='mps')

## Hyperparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE':512,
    'EPOCHS':15,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [ ]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [ ]:
# EDA
def data_preprocessing(dataframe):
    
    # drop_duplicates
    dataframe.drop_duplicates(inplace=True)
    
    # drop non-training columns
    dataframe.drop(['DCIS_or_LCIS_type', 'HER2_SISH', 'HER2_SISH_ratio'], axis = 1, inplace=True)
    
    # fillna BRCA_mutation, 3 => '3' is new category that means 'not examine'
    dataframe['BRCA_mutation'].fillna(3.0, inplace=True)
    
    # data processing about 'HG' columns
    for i in range(len(dataframe)):
        # if every values about 'HG' is null that fill in grade '4'
        if (np.isnan(dataframe.loc[i, 'HG'])) & (np.isnan(dataframe.loc[i, 'HG_score_1'])) & (np.isnan(dataframe.loc[i, 'HG_score_2'])) & (np.isnan(dataframe.loc[i, 'HG_score_3'])):
            dataframe.loc[i, 'HG'] = 4.0
            dataframe.loc[i, 'HG_score_1'] = 4.0
            dataframe.loc[i, 'HG_score_2'] = 4.0
            dataframe.loc[i, 'HG_score_3'] = 4.0
        # if every values about 'HG_score' except 'HG' is null that fill in condition
        elif (not np.isnan(dataframe.loc[i, 'HG'])) & (np.isnan(dataframe.loc[i, 'HG_score_1'])) & (np.isnan(dataframe.loc[i, 'HG_score_2'])) & (np.isnan(dataframe.loc[i, 'HG_score_3'])):
            # if 'HG' is 2.0 that fill in 2.0
            if dataframe.loc[i, 'HG'] == 2.0:
                dataframe.loc[i, 'HG_score_1'] = 2.0
                dataframe.loc[i, 'HG_score_2'] = 2.0
                dataframe.loc[i, 'HG_score_3'] = 2.0
            # if 'HG' is 3.0 that fill in 3.0
            elif dataframe.loc[i, 'HG'] == 3.0:
                dataframe.loc[i, 'HG_score_1'] = 3.0
                dataframe.loc[i, 'HG_score_2'] = 3.0
                dataframe.loc[i, 'HG_score_3'] = 3.0
            else:
                dataframe.loc[i, 'HG_score_1'] = 1.0
                dataframe.loc[i, 'HG_score_2'] = 1.0
                dataframe.loc[i, 'HG_score_3'] = 1.0
        # other 'HG' cases drop index at the end of EDA
    
    # data processing about 'NG' columns
    for f in range(len(dataframe)):
        if (np.isnan(dataframe.loc[f, 'NG'])) & (dataframe.loc[f, 'HG'] == 1.0) | (dataframe.loc[f, 'HG'] == 4.0):
            dataframe.loc[f, 'NG'] = 1.0
        elif (np.isnan(dataframe.loc[f, 'NG'])) & (dataframe.loc[f, 'HG'] == 2.0):
            dataframe.loc[f, 'NG'] = 2.0
        elif (np.isnan(dataframe.loc[f, 'NG'])) & (dataframe.loc[f, 'HG'] == 3.0):
            dataframe.loc[f, 'NG'] = 3.0
    
    # 'T_category' column fill in mode
    dataframe['T_category'].fillna(dataframe['T_category'].mode()[0], inplace=True)
    
    # 'KI-67' column fill in mean
    dataframe['KI-67_LI_percent'].fillna(dataframe['KI-67_LI_percent'].mean(), inplace=True)
    
    # if 'ER' or 'PR' is null that fill in condition
    for g in range(len(dataframe)):
        if (np.isnan(dataframe.loc[g, 'ER'])) | (np.isnan(dataframe.loc[g, 'PR'])):
            dataframe.drop(g, inplace=True, axis = 0)
    dataframe.reset_index(drop=True, inplace=True)

    for j in range(len(dataframe)):
        if (dataframe.loc[j, 'ER'] == 0.0) & (np.isnan(dataframe.loc[j, 'ER_Allred_score'])):
            dataframe.loc[j, 'ER_Allred_score'] = 0.0
        elif (dataframe.loc[j, 'ER'] == 1.0) & (np.isnan(dataframe.loc[j, 'ER_Allred_score'])):
            dataframe.loc[j, 'ER_Allred_score'] = dataframe['ER'].mean()

    for d in range(len(dataframe)):
        if (dataframe.loc[d, 'PR'] == 0.0) & (np.isnan(dataframe.loc[d, 'PR_Allred_score'])):
            dataframe.loc[d, 'PR_Allred_score'] = 0.0
        elif (dataframe.loc[d, 'PR'] == 1.0) & (np.isnan(dataframe.loc[d, 'PR_Allred_score'])):
            dataframe.loc[d, 'PR_Allred_score'] = dataframe['PR'].mean()
    
    # '암의 장경' fill in mean
    dataframe['암의 장경'].fillna(dataframe['암의 장경'].mean(), inplace=True)

    # 'HER2_IHC' fill in contion
    for h in range(len(dataframe)):
        if np.isnan(dataframe.loc[h, 'HER2_IHC']):
            dataframe.loc[h, 'HER2_IHC'] = dataframe.loc[h, 'HER2']
        elif (dataframe.loc[h, 'HER2_IHC'] < 2) & (dataframe.loc[h, 'HER2'] == 1):
            dataframe.loc[h, 'HER2_IHC'] = 2.0
        elif (dataframe.loc[h, 'HER2_IHC'] > 1) & (dataframe.loc[h, 'HER2'] == 0):
            dataframe.loc[h, 'HER2_IHC'] = 1.0
        elif np.isnan(dataframe.loc[h, 'HER2']):
            if dataframe.loc[h, 'HER2_IHC'] >= 2:
                dataframe.loc[h, 'HER2'] = 1.0
            elif dataframe.loc[h, 'HER2_IHC'] < 2:
                dataframe.loc[h, 'HER2'] = 0.0
    
    # drop another 'NaN' values
#     dataframe = dataframe.fillna(0) #추가됨
    dataframe.dropna(inplace = True)
    dataframe.reset_index(drop=True, inplace=True)

    print(len(dataframe))
    print(dataframe.isnull().sum())
    return dataframe

In [ ]:
train_df = data_preprocessing(train_df)
test_df = data_preprocessing(test_df)

996
ID                  0
img_path            0
mask_path           0
나이                  0
수술연월일               0
진단명                 0
암의 위치               0
암의 개수               0
암의 장경               0
NG                  0
HG                  0
HG_score_1          0
HG_score_2          0
HG_score_3          0
DCIS_or_LCIS_여부     0
T_category          0
ER                  0
ER_Allred_score     0
PR                  0
PR_Allred_score     0
KI-67_LI_percent    0
HER2                0
HER2_IHC            0
BRCA_mutation       0
N_category          0
dtype: int64
250
ID                  0
img_path            0
나이                  0
수술연월일               0
진단명                 0
암의 위치               0
암의 개수               0
암의 장경               0
NG                  0
HG                  0
HG_score_1          0
HG_score_2          0
HG_score_3          0
DCIS_or_LCIS_여부     0
T_category          0
ER                  0
ER_Allred_score     0
PR                  0
PR_Allred_score     0
KI-67_LI_pe

In [ ]:
train_df['img_path'] = train_df['img_path'].str.replace('\./', './data/')
test_df['img_path'] = test_df['img_path'].str.replace('\./', './data/')

train_df.head(3)

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,T_category,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,BRCA_mutation,N_category
0,BC_01_0001,./data/train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.000000,2.0,...,1.0,1.0,8.0,1.0,6.0,12.0,0.0,1.0,3.0,0
1,BC_01_0002,./data/train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.000000,3.0,...,2.0,0.0,0.0,0.0,0.0,70.0,0.0,0.0,3.0,1
2,BC_01_0003,./data/train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,18.375564,1.0,...,0.0,1.0,7.0,1.0,4.0,7.0,0.0,1.0,0.0,0


In [ ]:
train_df, val_df, train_labels, val_labels = train_test_split(
                                                    train_df.drop(columns=['N_category']), 
                                                    train_df['N_category'], 
                                                    test_size=0.2, 
                                                    random_state=CFG['SEED']
                                                )

In [ ]:
def get_values(value):
    return value.values.reshape(-1, 1)

numeric_cols = ['나이', '암의 장경', 'ER_Allred_score', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2_SISH_ratio']
ignore_cols = ['ID', 'img_path', 'mask_path', '수술연월일', 'N_category']

for col in train_df.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        scaler = StandardScaler()
        train_df[col] = scaler.fit_transform(get_values(train_df[col]))
        val_df[col] = scaler.transform(get_values(val_df[col]))
        test_df[col] = scaler.transform(get_values(test_df[col]))
    else:
        le = LabelEncoder()
        train_df[col] = le.fit_transform(get_values(train_df[col]))
        val_df[col] = le.transform(get_values(val_df[col]))
        test_df[col] = le.transform(get_values(test_df[col]))

## CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, medical_df, labels, transforms=None):
        self.medical_df = medical_df
        self.transforms = transforms
        self.labels = labels
        
    def __getitem__(self, index):
        img_path = self.medical_df['img_path'].iloc[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
                
        if self.labels is not None:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', 'mask_path', '수술연월일']).iloc[index])
            label = self.labels[index]
            return image, tabular, label
        else:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', '수술연월일']).iloc[index])
            return image, tabular
        
    def __len__(self):
        return len(self.medical_df)

In [ ]:
train_transforms = A.Compose([
                            A.HorizontalFlip(),
                            A.VerticalFlip(),
                            A.Rotate(limit=90, border_mode=cv2.BORDER_CONSTANT,p=0.3),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transforms = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(train_df, train_labels.values, train_transforms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df, val_labels.values, test_transforms)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Architecture

In [ ]:
class ImgFeatureExtractor(nn.Module):
    def __init__(self):
        super(ImgFeatureExtractor, self).__init__()
        model = models.efficientnet_b0(pretrained=True)
    
        for param in model.features.parameters():
            param.requires_grad = False
            
        # remove last SqueezeExcitation
        t_model = list(model.children())[0][7][0]
        t_MBConv = list(t_model.children())[0]
        tmp = t_MBConv
        del tmp[2]
        self.backbone = model

        self.embedding = nn.Linear(1000,512)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.embedding(x)
        return x

In [ ]:
class TabularFeatureExtractor(nn.Module):
    def __init__(self):
        super(TabularFeatureExtractor, self).__init__()
        self.embedding = nn.Sequential(
            nn.Linear(in_features=20, out_features=32),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Linear(in_features=32, out_features=64),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.Linear(in_features=64, out_features=128),
            nn.BatchNorm1d(128),
            nn.ELU(),
            nn.Linear(in_features=128, out_features=256),
            nn.BatchNorm1d(256),
            nn.ELU(),
            nn.Linear(in_features=256, out_features=512),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.Linear(in_features=512, out_features=512)
        )
        
    def forward(self, x):
        x = self.embedding(x)
        return x

In [ ]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.img_feature_extractor = ImgFeatureExtractor()
        self.tabular_feature_extractor = TabularFeatureExtractor()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=640, out_features=1),
            nn.Sigmoid(),
        )
        
    def forward(self, img, tabular):
        img_feature = self.img_feature_extractor(img)
        tabular_feature = self.tabular_feature_extractor(tabular)
        feature = torch.cat([img_feature, tabular_feature], dim=-1)
        output = self.classifier(feature)
        return output

## Train

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for img, tabular, label in tqdm(iter(train_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            label = label.float().to(device)
            
            optimizer.zero_grad()
            
            model_pred = model(img, tabular)
            
            loss = criterion(model_pred, label.reshape(-1,1))
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        val_loss, val_score = validation(model, criterion, val_loader, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_score)
        
        if best_score < val_score:
            best_score = val_score
            best_model = model
    
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    pred_labels = []
    true_labels = []
    val_loss = []
    threshold = 0.5
    with torch.no_grad():
        for img, tabular, label in tqdm(iter(val_loader)):
            true_labels += label.tolist()
            
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            label = label.float().to(device)
            
            model_pred = model(img, tabular)
            
            loss = criterion(model_pred, label.reshape(-1,1))
            
            val_loss.append(loss.item())
            
            model_pred = model_pred.squeeze(1).to('cpu')  
            pred_labels += model_pred.tolist()
    
    pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
    val_score = metrics.f1_score(y_true=true_labels, y_pred=pred_labels, average='macro')
    return np.mean(val_loss), val_score

## Train

In [ ]:
model = nn.DataParallel(ClassificationModel())
model.eval()

DataParallel(
  (module): ClassificationModel(
    (img_feature_extractor): ImgFeatureExtractor(
      (backbone): EfficientNet(
        (features): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): Sequential(
            (0): MBConv(
              (block): Sequential(
                (0): Conv2dNormActivation(
                  (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (2): SiLU(inplace=True)
                )
                (1): SqueezeExcitation(
                  (avgpool): AdaptiveAvgPool2d(output_size=1)
                  (fc1): Conv2d(32, 8, kernel_size=(1, 1

In [ ]:

optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.66007] Val Loss : [0.64533] Val Score : [0.76870]


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

Epoch [14], Train Loss : [0.58265] Val Loss : [0.61465] Val Score : [0.75830]
Epoch 00014: reducing learning rate of group 0 to 6.2500e-06.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

Epoch [15], Train Loss : [0.58078] Val Loss : [0.61390] Val Score : [0.79954]


## Inference

In [ ]:
test_dataset = CustomDataset(test_df, None, test_transforms)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    threshold = 0.5
    
    with torch.no_grad():
        for img, tabular in tqdm(iter(test_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            
            model_pred = model(img, tabular)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            
            preds += model_pred.tolist()
    
    preds = np.where(np.array(preds) > threshold, 1, 0)
    
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/16 [00:00<?, ?it/s]